In [44]:
import ps
import ps_utils
import datetime as dt
import pandas as pd
import numpy as np
import bisect

ps_utils.switch_to_dev()
pricing_date = dt.date(2023, 8, 17)
pricing_date

[INFO] 2023-08-31 17:51:31,870 - Nothing to do: current environment is already dev


datetime.date(2023, 8, 17)

In [45]:
mds = ps_utils.get_mds(pricing_date)
m_x = [(x.date()-pricing_date).days/365. for x in mds.RatesCurvesBundles.RUB_RUONIA_OIS.RatesSchedule.Dates]
m_y = [y for y in mds.RatesCurvesBundles.RUB_RUONIA_OIS.RatesSchedule.Rates]
pd.reset_option('display.float_format')
pd.set_option("display.precision", 3)
pd.DataFrame([m_x, m_y])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.003,0.022,0.041,0.088,0.170,0.260,0.510,0.759,1.008,2.005,3.005,4.005,5.008,6.014,7.011,8.008,9.011,10.011,12.016,15.014
1,11.768,11.959,11.959,11.934,11.879,11.603,11.137,10.874,10.741,10.505,10.391,10.339,10.297,10.252,10.221,10.190,10.174,10.164,10.136,10.193


In [46]:
def cubic_hermite(x, m_x, m_y, deriv_method = 'Bessel', left_extrap = 'Flat', right_extrap = 'Flat'):
    # only flat extrapolation at this stage
    if x <= m_x[0]:
        if left_extrap == 'Flat':
            return m_y[0]
        else:
            raise ValueError('left extrapolation type '+left_extrap+' is not supported')
    elif x >= m_x[-1]:
        if right_extrap == 'Flat':
            return m_y[-1]
        else:
            raise ValueError('right extrapolation type '+right_extrap+' is not supported')        
    # get the index of the previous point
    i = bisect.bisect_right(m_x, x)-1
    x0 = m_x[i]
    x1 = m_x[i+1]
    y0 = m_y[i]
    y1 = m_y[i+1]
    tan = (y1-y0)/(x1-x0)
    # === here we set derivatives in the pillars according to Bessel method
    # === there can be other approaches as well
    if deriv_method == 'Bessel':
        # get left derivative
        if i == 0:
            z0 = tan
        else:
            tan_prev = (y0-m_y[i-1])/(x0-m_x[i-1])
            z0 = ((x0-m_x[i-1])*tan+(x1-x0)*tan_prev)/(x1-m_x[i-1])
        # get right derivative
        if i == len(m_x) -2:
            z1 = tan
        else:
            tan_next = (m_y[i+2]-y1)/(m_x[i+2]-x1)
            z1 = ((m_x[i+2]-x1)*tan+(x1-x0)*tan_next)/(m_x[i+2]-x0)
    else:
        raise ValueError('method '+deriv_method+' is not supported') 
    # yf coefficients
    t = (x-x0)/(x1-x0)
    t3 = t**3.
    t2 = t**2.
    h00 = 2.*t3 - 3.*t2 + 1.
    h10 = t3 - 2.*t2 + t
    h01 = -2.*t3 + 3.*t2
    h11 = t3 - t2
    # calculate
    return h00*y0 + h10*z0*(x1-x0) + h01*y1 + h11*z1*(x1-x0)

## Test versus Murex

In [47]:
# murex_rates_pd = pd.read_csv('hermite_rates_170823.csv')
# murex_rates = ps.put(ps.new_cm({'yf': murex_rates_pd['yf'].to_list(),'rates':murex_rates_pd['rates'].to_list()}))
murex_rates = ps.get('8e03418ffcb3d51a3951e60c2414a3e8fa7dc8fc')

In [49]:
recalc_rates = []
for yf in murex_rates['yf']:
    recalc_rates += [cubic_hermite(yf,m_x, m_y, deriv_method = 'Bessel')]

In [50]:
result = pd.DataFrame(index=murex_rates['yf'])
result['mx_rates'] = murex_rates['rates']
result['recalc_rates']=recalc_rates
result['diff'] = result['recalc_rates']-result['mx_rates']
(result['diff'].min(),result['diff'].max())

(-3.47455397786689e-12, 1.0835776720341528e-13)